In [1]:
import re
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [3]:
def getRegex(df : pd.DataFrame, col_name : str, regex : str) -> pd.DataFrame :
    return df[df[col_name].str.contains(regex)]

# 전방탐색과 후방탐색

* 어디서 텍스트를 찾을지를 표시하는데 표현식을 사용
* 전후방탐색 (lookaroung)

## 전후방탐색 살펴보기

In [4]:
text1 = f"""<head>
<title>Ben Forta`s Homepage</title>
</head>"""

In [6]:
print(re.search(r"<[Tt][Ii][Tt][Ll][Ee]>.*<\/[Tt][Ii][Tt][Ll][Ee]>", text1))

<re.Match object; span=(7, 42), match='<title>Ben Forta`s Homepage</title>'>


## 전방탐색(lookahead)

* 전방탐색이란? 
    * 패턴의 일치 영역을 발견해도 그 값을 반환하지 않는 패턴
    * 실제로는 하위 표현식이며 하위 표현식과 같은 형태로 작성
    * ?= 다음에 일치할 텍스트가 오는 하위 표현식

In [7]:
text2 = f"""http://www.forta.com/
https://mail.forta.com/
ftp://ftp.forta.com/"""

In [8]:
print(re.findall(r".+(?=:)", text2))

['http', 'https', 'ftp']


* 만약 전방 탐색을 사용하지 않는다면?

In [10]:
print(re.findall(r"(.+(:))", text2))

[('http:', ':'), ('https:', ':'), ('ftp:', ':')]


* 전후방탐색 일치는 실제로 결과를 반환하지만, 반환된 문자의 길이가 항상0이다. 따라서, 전방탐색을 흔히 제로 폭이라 부르기도 한다.

## 후방탐색(lookbehind)

* ?<= : 뒤쪽을 탐색
* 하위 표현식 안에서 사용, 일치할 텍스트 앞에 위치

In [11]:
text3 = f"""ABC01: $23.45
HGG42: $5.31
CFMX1:$899.00
XTC99: $69.96
Total items found: 4"""

In [12]:
print(re.findall(r"\$[0-9.]+", text3))

['$23.45', '$5.31', '$899.00', '$69.96']


* 만약 달러를 패턴에서 제외한다면?

In [13]:
print(re.findall(r"[0-9.]+", text3))

['01', '23.45', '42', '5.31', '1', '899.00', '99', '69.96', '4']


* 하지만 후방 탐색을 사용한다면?

In [15]:
print(re.findall(r"(?<=\$)[0-9.]+", text3))

['23.45', '5.31', '899.00', '69.96']


* 전방탐색 패턴은 마침표와 더하기를 포함하여 텍스트의 길이를 다양하게 일치시킬 수 있으며, 매우 동적이다.
* 반대로 후방탐색 패턴은 보통 일치시킬 텍스트의 길이를 고정해야 한다. 
* 거의 모든 정규 표현식 구현에는 이런 제약이 있다.

## 전방탐색과 후방탐색 함께 사용하기

In [19]:
print(re.search(r"(?<=<[Tt][Ii][Tt][Ll][Ee]>).*(?=<\/[Tt][Ii][Tt][Ll][Ee]>)", text1))

<re.Match object; span=(14, 34), match='Ben Forta`s Homepage'>


## 부정형 전후방탐색

* 지금까지 살펴봤듯이 후방탐색과 전방탐색은 반환할 텍스트의 위치, 즉 찾고자 하는 부분의 앞뒤를 특별히 지정하고 싶을 때 주로 사용한다.
* 이런 방법들을 긍정형 전방탐색과 긍정형 후방탐색이라고 한다.
    * 긍정형이라는 단어가 붙은 이유는 실제로 일치하는 텍스트를 찾기 때문이다.
* 반대로 부정형은 실제로 일치하지 않는 단어를 찾는다.
    * (?=) : 긍정형 전방탐색
    * (?!) : 부정형 전방탐색
    * (?<=) : 긍정형 후방탐색
    * (?<!) : 부정형 후방탐색
* 부정형은 등호 대신에 느낌표를 사용한다.

In [20]:
text4 = f"""I paid $30 for 100 apples,
50 oranges, and 60 pears.
I saved $5 on this order."""

In [22]:
#가격만 얻기
print(re.findall(r"(?<=\$)\d+", text4))

['30', '5']


In [25]:
#수량만 얻기
print(re.findall(r"\b(?<!\$)\d+\b", text4))

['100', '50', '60']


* 정확히 수량만 얻어왔다.
* 만약, 경계 \b를 사용하지 않았다면?

In [27]:
#수량만 얻기
print(re.findall(r"(?<!\$)\d+", text4))

['0', '100', '50', '60']


* 보시다시피 $30에서 0도 인식된다. 왜냐하면, 0앞에 달러 기호가 엎기 때문이다 .따라서, 경계 조건을 사용한 것이다.

* **전후방 탐색을 사용하는 이유는 무엇을 반환하는지 더 구체적으로 명시하는 것이다.**
* **하위 표현식을 통해 텍스트의 위치는 지정하지만, 소비하지는 않는다.**